In [1]:
from minimalist_parser.algebras.string_algebra import BareTreeStringAlgebra
from minimalist_parser.algebras.algebra import AlgebraOp, AlgebraTerm
from minimalist_parser.minimalism.minimalist_algebra_synchronous import MinimalistAlgebraSynchronous, MinimalistFunctionSynchronous, SynchronousTerm, InnerAlgebraInstructions


We start by initialising an algebra over strings with operation names < and >. The terms over this algebra are thus Bare Trees a la Stabler 1997.

In [2]:
string_algebra = BareTreeStringAlgebra()

Algebra constants are zeroary operations. They have a name and a zeroary function, which is an element of the domain: here, a string.

In [3]:
the = AlgebraOp("the", "the")

# The algebra also has a default constant maker
puppy = string_algebra.constant_maker("puppy")
snuggled = string_algebra.constant_maker("snuggled")

# look at the vocabulary
vocab = [the, puppy, snuggled]
for algebra_op in vocab:
    print("\nname:", algebra_op.name)
    print("function:", algebra_op.function)



name: the
function: the

name: puppy
function: puppy

name: snuggled
function: snuggled


Binary operations < and > can be looked up in `string_algebra.ops` or they can be created. 

For consistency across Head Movement Algebras, < is in `string_algebra.ops['concat_right']` and > is in `string_algebra.ops['concat_left']`. Notice both use the string_algebra.concat_right method, since Bare Trees are WYSIWYG when it comes to word order.

In [4]:
print(string_algebra.ops['concat_right'].name)
print(string_algebra.ops['concat_right'].function)
print(string_algebra.ops['concat_left'].name)
print(string_algebra.ops['concat_left'].function)

<
<bound method BareTreeStringAlgebra.concat_right of String Algebra>
>
<bound method BareTreeStringAlgebra.concat_right of String Algebra>


You can always just create an AlgebraOp as well, if you want.

In [5]:
new_right = AlgebraOp("<", string_algebra.concat_right)
print(new_right == string_algebra.ops['concat_right'])

True


### Algebra Terms

An term over an algebra is a tree in which the nodes are labelled with operations of that algebra, and the number of children of the node matches the arity of the function of the operation.

This means the leaves are constants, and the internal nodes are operations. 

An `AlgebraTerm` has a `parent`, which is an `AlgebraOp`, and, optionally, a list of `children`, which are `AlgebraTerm`s.

In [8]:
# for ease of typing, let's give < and > names
head_left = string_algebra.ops['concat_right']
head_right = string_algebra.ops['concat_left']

# we also really want our vocabulary to be AlgebraTerms. We can turn them all into AlgebraTerms, but instead let's make them with the convenience function make_leaf
the = string_algebra.make_leaf("the")
puppy = string_algebra.make_leaf("puppy")
snuggled = string_algebra.make_leaf("snuggled")

the_puppy = AlgebraTerm(head_left, [the, puppy])

print(the_puppy)

('<', [('the'), ('puppy')])


These can be exported to nltk Trees, which allows us to visualise them.

In [9]:
the_puppy.to_nltk_tree().draw()

In [10]:
the_puppy_snuggled = AlgebraTerm(head_right, [the_puppy, snuggled])
the_puppy_snuggled.to_nltk_tree().draw()

## Minimalist Algebras

A Minimalist Algebra is also an algebra, but it's quite different from a string-building algebra. A Minimalist Algebra essentially handles all the Move-related work. Structure-building is delegated to its "inner algebra", which can be, for instance, this `string_algebra`.

The domain of a Minimalist Algebra is `Expression`s, which contain an `inner_term` (a term of the inner algebra), `Movers`, and have an `mg_type` with things like +/- lexical and +/- conjunction.

`Movers` implements a partial function from slot names (such as `'-wh'` or `'Abar'`) to inner terms. 

Today we'll just look at the `MinimalistAlgebraSynchronous` subclass, so I can show you how to build a synchronous grammar over multiple inner algebras.

In [11]:
# a MinimalistAlgebraSynchronous requires a list of inner algebras.
minimalist_algebra = MinimalistAlgebraSynchronous([string_algebra])

In [12]:
print(minimalist_algebra.ops)

{}


In [13]:
minimalist_algebra.add_default_operations()

In [14]:
print(minimalist_algebra.ops)

{'merge1+concat_right': merge1+concat_right, 'merge1+concat_left': merge1+concat_left, 'merge2+A': merge2+A, 'move1+A+concat_right': move1+A+concat_right, 'move1+A+concat_left': move1+A+concat_left, 'move2+A+A': move2+A+A, 'move2+A+ABar': move2+A+ABar, 'move2+A+R': move2+A+R, 'move2+A+Self': move2+A+Self, 'merge2+ABar': merge2+ABar, 'move1+ABar+concat_right': move1+ABar+concat_right, 'move1+ABar+concat_left': move1+ABar+concat_left, 'move2+ABar+A': move2+ABar+A, 'move2+ABar+ABar': move2+ABar+ABar, 'move2+ABar+R': move2+ABar+R, 'move2+ABar+Self': move2+ABar+Self, 'merge2+R': merge2+R, 'move1+R+concat_right': move1+R+concat_right, 'move1+R+concat_left': move1+R+concat_left, 'move2+R+A': move2+R+A, 'move2+R+ABar': move2+R+ABar, 'move2+R+R': move2+R+R, 'move2+R+Self': move2+R+Self, 'merge2+Self': merge2+Self, 'move1+Self+concat_right': move1+Self+concat_right, 'move1+Self+concat_left': move1+Self+concat_left, 'move2+Self+A': move2+Self+A, 'move2+Self+ABar': move2+Self+ABar, 'move2+Self+R':

In [15]:
from minimalist_parser.algebras.hm_algebra import HMAlgebra


In [122]:
from minimalist_parser.algebras.algebra_objects.fake_set import FakeSet


def make_fake_set_constant(word, label):
    """
    make an AlgebraOp with the given word as the content of a unary FakeSet.
    With this we can use the synchronous algebra make_leaf function for a shortcut to a term leaf.
    """
    return AlgebraOp(label, FakeSet([word]))

set_algebra = HMAlgebra("set algebra", FakeSet)
set_algebra.add_constant_maker(make_fake_set_constant)

In [123]:
set_algebra.concat_right([FakeSet([7,8,1]), FakeSet([3, 2])]).spellout()

{{1, 7, 8}, {2, 3}}

In [124]:
minimalist_algebra = MinimalistAlgebraSynchronous([string_algebra, set_algebra])

In [125]:
minimalist_algebra.add_default_operations()

In [126]:
print(minimalist_algebra.ops)

{'merge1+concat_right+concat_right': merge1+concat_right+concat_right, 'merge1+concat_left+concat_left': merge1+concat_left+concat_left, 'merge2+A': merge2+A, 'move1+A+concat_right+concat_right': move1+A+concat_right+concat_right, 'move1+A+concat_left+concat_left': move1+A+concat_left+concat_left, 'move2+A+A': move2+A+A, 'move2+A+ABar': move2+A+ABar, 'move2+A+R': move2+A+R, 'move2+A+Self': move2+A+Self, 'merge2+ABar': merge2+ABar, 'move1+ABar+concat_right+concat_right': move1+ABar+concat_right+concat_right, 'move1+ABar+concat_left+concat_left': move1+ABar+concat_left+concat_left, 'move2+ABar+A': move2+ABar+A, 'move2+ABar+ABar': move2+ABar+ABar, 'move2+ABar+R': move2+ABar+R, 'move2+ABar+Self': move2+ABar+Self, 'merge2+R': merge2+R, 'move1+R+concat_right+concat_right': move1+R+concat_right+concat_right, 'move1+R+concat_left+concat_left': move1+R+concat_left+concat_left, 'move2+R+A': move2+R+A, 'move2+R+ABar': move2+R+ABar, 'move2+R+R': move2+R+R, 'move2+R+Self': move2+R+Self, 'merge2+Sel

In [127]:
merge_right = minimalist_algebra.ops["merge1+concat_right+concat_right"]

In [128]:
the = minimalist_algebra.make_leaf("the")
puppy = minimalist_algebra.make_leaf("puppy")

silent_inners = {inner_algebra: InnerAlgebraInstructions(op_name="[past]", leaf_object=inner_algebra.domain_type()) for inner_algebra in minimalist_algebra.inner_algebras}

past = minimalist_algebra.make_leaf("[past]", silent_inners)

print(past.interp(set_algebra).inner_term.evaluate())

t=SynchronousTerm(merge_right, [past, SynchronousTerm(merge_right, [the, puppy])])

{}


In [130]:
print(t.spellout(set_algebra))
t.spellout(string_algebra)

{{}, {{puppy}, {the}}}


'the puppy'

In [29]:
FakeSet()

{}